In [8]:
class EWMALoopBoundDetector:
    def __init__(self, alpha=0.5, initial_bound=10):
        """
        初始化EWMA循环边界检测器。
        
        :param alpha: EWMA中的平滑系数，控制新值和历史值的重要性。取值范围为[0, 1]。
        :param initial_bound: 初始的循环边界估计值。
        """
        self.alpha = alpha
        self.estimated_bound = initial_bound  # 初始化的循环边界估计
        self.iteration_count = 0  # 当前循环的迭代次数

    def update(self, actual_iteration_count):
        """
        更新当前循环边界的估计值，使用实际的迭代次数。
        
        :param actual_iteration_count: 当前循环实际执行的迭代次数。
        :return: 更新后的边界估计值。
        """
        # 使用EWMA公式更新估计的边界
        self.estimated_bound = self.alpha * actual_iteration_count + (1 - self.alpha) * self.estimated_bound
        self.iteration_count = actual_iteration_count
        return self.estimated_bound

    def get_estimated_bound(self):
        """
        获取当前的循环边界估计值。
        
        :return: 当前估计的循环边界值。
        """
        return self.estimated_bound


# 模拟使用EWMA循环边界检测器
if __name__ == "__main__":
    # 初始化EWMA检测器，平滑系数为0.5
    lbd = EWMALoopBoundDetector(alpha=0.3, initial_bound=10)

    # 假设我们有一个循环，其实际的迭代次数在变化
    actual_iterations = [12, 15, 14, 16, 20, 18, 22]

    print("Initial Estimated Loop Bound:", lbd.get_estimated_bound())

    # 模拟多次循环迭代并更新EWMA边界估计
    for i, actual in enumerate(actual_iterations):
        updated_bound = lbd.update(actual)
        print(f"Iteration {i + 1}: Actual Iterations = {actual}, Updated Estimated Bound = {updated_bound:.2f}")


Initial Estimated Loop Bound: 10
Iteration 1: Actual Iterations = 12, Updated Estimated Bound = 10.60
Iteration 2: Actual Iterations = 15, Updated Estimated Bound = 11.92
Iteration 3: Actual Iterations = 14, Updated Estimated Bound = 12.54
Iteration 4: Actual Iterations = 16, Updated Estimated Bound = 13.58
Iteration 5: Actual Iterations = 20, Updated Estimated Bound = 15.51
Iteration 6: Actual Iterations = 18, Updated Estimated Bound = 16.25
Iteration 7: Actual Iterations = 22, Updated Estimated Bound = 17.98


In [9]:
class LoopBoundDetectorGroup:
    """循环边界检测器组，包含多个EWMA循环边界检测器"""
    
    def __init__(self, num_detectors=16, alpha=0.5, initial_bound=10):
        """
        初始化LBD组，每个检测器有独立的EWMA参数。
        
        :param num_detectors: 检测器的数量
        :param alpha: EWMA中的平滑系数
        :param initial_bound: 每个LBD的初始循环边界值
        """
        self.detectors = [EWMALoopBoundDetector(alpha=alpha, initial_bound=initial_bound) for _ in range(num_detectors)]

    def update_group(self, actual_iterations_list):
        """
        更新整个检测器组的边界估计值，每个检测器根据实际的迭代次数更新。
        
        :param actual_iterations_list: 每个检测器对应的实际迭代次数列表
        :return: 包含每个检测器的更新后估计边界的结果列表
        """
        assert len(actual_iterations_list) == len(self.detectors), "实际迭代次数列表的长度应与检测器数量相等"
        
        results = []
        for i, actual_iterations in enumerate(actual_iterations_list):
            updated_bound = self.detectors[i].update(actual_iterations)
            results.append((i, actual_iterations, updated_bound))
        
        return results

    def get_estimated_bounds(self):
        """
        获取所有检测器的当前边界估计值。
        
        :return: 每个检测器的估计边界值列表
        """
        return [detector.get_estimated_bound() for detector in self.detectors]


if __name__ == "__main__":
    # 初始化LBD组，包含4个检测器，平滑系数为0.5，初始边界为10
    lbd_group = LoopBoundDetectorGroup(num_detectors=4, alpha=0.3, initial_bound=10)

    # 模拟多个LBD对应的实际迭代次数
    actual_iterations_group = [
        [12, 15, 14, 16],  # 第一次四个检测器的实际迭代次数
        [20, 18, 22, 21],  # 第二次四个检测器的实际迭代次数
        [30, 28, 32, 29],  # 第三次四个检测器的实际迭代次数
        [40, 42, 44, 41],  # 第四次四个检测器的实际迭代次数
    ]

    # 打印初始边界估计值
    print("Initial Estimated Bounds:", lbd_group.get_estimated_bounds())

    # 更新每个LBD，并打印结果
    for i, actual_iterations in enumerate(actual_iterations_group):
        updated_bounds = lbd_group.update_group(actual_iterations)
        print(f"{i + 1} times Group Updated Bounds:")
        for detector_id, actual, bound in updated_bounds:
            print(f"  Detector {detector_id}: Actual Iterations = {actual}, Updated Estimated Bound = {bound:.2f}")

Initial Estimated Bounds: [10, 10, 10, 10]
1 times Group Updated Bounds:
  Detector 0: Actual Iterations = 12, Updated Estimated Bound = 10.60
  Detector 1: Actual Iterations = 15, Updated Estimated Bound = 11.50
  Detector 2: Actual Iterations = 14, Updated Estimated Bound = 11.20
  Detector 3: Actual Iterations = 16, Updated Estimated Bound = 11.80
2 times Group Updated Bounds:
  Detector 0: Actual Iterations = 20, Updated Estimated Bound = 13.42
  Detector 1: Actual Iterations = 18, Updated Estimated Bound = 13.45
  Detector 2: Actual Iterations = 22, Updated Estimated Bound = 14.44
  Detector 3: Actual Iterations = 21, Updated Estimated Bound = 14.56
3 times Group Updated Bounds:
  Detector 0: Actual Iterations = 30, Updated Estimated Bound = 18.39
  Detector 1: Actual Iterations = 28, Updated Estimated Bound = 17.81
  Detector 2: Actual Iterations = 32, Updated Estimated Bound = 19.71
  Detector 3: Actual Iterations = 29, Updated Estimated Bound = 18.89
4 times Group Updated Bound